# Get one symbol nakeds for `NSE`
***

- [x] get equity fno list
- [x] get equity histories
- [x] get index histories
- [x] get lot-size
- [x] get market price of options
- [x] get volatilities for chains
- [x] get option closest to underlying
- [x] get all chains with dte

***

- [ ] get rim SDs based on SDMULT and dte
- [ ] get margins
- [ ] get equity option histories
- [ ] get index option histories

***
 - [ ] pack all into symbol objects



In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

# Imports

In [ ]:
import json
import math
from datetime import date, datetime, time, timedelta, timezone

import numpy as np
import pytz
import requests
from pandas import json_normalize
from tqdm import tqdm

from utils import create_dataclass_from_dict

# Constants

In [ ]:
PUTSTDMULT = 2
CALLSTDMULT = 3

# Helper functions

In [ ]:
# HELPER FUNCTIONS
# ****************


def live_cache(app_name):
    """Caches the output for time_out specified. This is done in order to
    prevent hitting live quote requests to NSE too frequently. This wrapper
    will fetch the quote/live result first time and return the same result for
    any calls within 'time_out' seconds.

    Logic:
        key = concat of args
        try:
            cached_value = self._cache[key]
            if now - self._cache['tstamp'] < time_out
                return cached_value['value']
        except AttributeError: # _cache attribute has not been created yet
            self._cache = {}
        finally:
            val = fetch-new-value
            new_value = {'tstamp': now, 'value': val}
            self._cache[key] = new_value
            return val

    """

    def wrapper(self, *args, **kwargs):
        """Wrapper function which calls the function only after the timeout,
        otherwise returns value from the cache.

        """
        # Get key by just concating the list of args and kwargs values and hope
        # that it does not break the code :P
        inputs = [str(a) for a in args] + [str(kwargs[k]) for k in kwargs]
        key = app_name.__name__ + "-".join(inputs)
        now = datetime.now()
        time_out = self.time_out
        try:
            cache_obj = self._cache[key]
            if now - cache_obj["timestamp"] < timedelta(seconds=time_out):
                return cache_obj["value"]
        except:
            self._cache = {}
        value = app_name(self, *args, **kwargs)
        self._cache[key] = {"value": value, "timestamp": now}
        return value

    return wrapper


def split_dates(days: int = 365, chunks: int = 50) -> list:
    """splits dates into buckets, based on chunks"""

    end = datetime.today()
    periods = int(days / chunks)
    start = end - timedelta(days=days)

    if days < chunks:
        date_ranges = [(start, end)]
    else:
        dates = pd.date_range(start, end, periods).date
        date_ranges = list(
            zip(pd.Series(dates), pd.Series(dates).shift(-1) + timedelta(days=-1))
        )[:-1]

    # remove last tuple having period as NaT
    if any(pd.isna(e) for element in date_ranges for e in element):
        date_ranges = date_ranges[:-1]

    return date_ranges


def make_date_range_for_stock_history(
    symbol: str, days: int = 365, chunks: int = 50
) -> list:
    """Uses `split_dates` to make date range for stock history"""

    date_ranges = split_dates(days=days, chunks=chunks)

    series = "EQ"

    ranges = [
        {
            "symbol": symbol,
            "from": start.strftime("%d-%m-%Y"),
            "to": end.strftime("%d-%m-%Y"),
            "series": f'["{series}"]',
        }
        for start, end in date_ranges
    ]

    return ranges


def clean_stock_history(result: list) -> pd.DataFrame:
    """Cleans output of"""

    df = pd.concat(
        [pd.DataFrame(r.get("data")) for r in result], axis=0, ignore_index=True
    )

    # ...clean columns

    mapping = {
        "CH_SYMBOL": "nse_symbol",
        "TIMESTAMP": "date",
        "CH_OPENING_PRICE": "open",
        "CH_TRADE_HIGH_PRICE": "high",
        "CH_TRADE_LOW_PRICE": "low",
        "CH_CLOSING_PRICE": "close",
        "CH_TOT_TRADED_QTY": "qty_traded",
        "CH_TOT_TRADED_VAL": "value_traded",
        "CH_TOTAL_TRADES": "trades",
        "VWAP": "vwap",
        "updatedAt": "extracted_on",
    }

    df = df[[col for col in mapping.keys() if col in df.columns]].rename(
        columns=mapping
    )

    # ...convert column datatypes

    astype_map = {
        **{
            k: "float"
            for k in ["open", "high", "low", "close", "value_traded", "trades", "vwap"]
        },
        **{"qty_traded": "int"},
    }

    df = df.astype(astype_map)

    # ...change date columns to utc

    replace_cols = ["date", "extracted_on"]
    df1 = df[replace_cols].map(lambda x: datetime.fromisoformat(x))
    df = df.assign(date=df1.date, extracted_on=df1.extracted_on)

    return df


def clean_index_history(results: list) -> pd.DataFrame:
    """cleans index history and builds it as a dataframe"""

    df = pd.concat(
        [pd.DataFrame(json.loads(r.get("d"))) for r in results], ignore_index=True
    )

    # clean the df

    # ...drop unnecessary columns

    df = df.drop(df.columns[[0, 1]], axis=1)

    # ...rename
    df.columns = ["nse_symbol", "date", "open", "high", "low", "close"]

    # ...convert nse_symbol to IB's symbol
    df = pd.concat(
        [
            df.nse_symbol.map(
                {"Nifty Bank": "BANKNIFTY", "Nifty 50": "NIFTY50"}
            ).rename("symbol"),
            df,
        ],
        axis=1,
    )

    utc_dates = dates_string_to_utc(df.date, eod=True)

    df = df.assign(date=utc_dates)

    # .....convert ohlc to numeric
    convert_dict = {k: "float" for k in ["open", "high", "low", "close"]}

    df = df.astype(convert_dict)

    # .....sort by date
    df.sort_values(["nse_symbol", "date"], inplace=True, ignore_index=True)

    # .....add extract_date
    now = datetime.now()
    utc_now = now.astimezone(timezone.utc)
    df = df.assign(extracted_on=utc_now)

    return df


def nse2ib(nse_list):
    """Converts nse to ib friendly symbols"""

    subs = {"M&M": "MM", "M&MFIN": "MMFIN", "L&TFH": "LTFH", "NIFTY": "NIFTY50"}

    list_without_percent_sign = list(map(subs.get, nse_list, nse_list))

    # fix length to 9 characters
    ib_equity_fnos = [s[:9] for s in list_without_percent_sign]

    return ib_equity_fnos


def convert_to_utc_datetime(date_string, eod=False):
    """Converts nse date strings to utc datetimes. If eod is chosen 3:30 PM IST is taken."""

    # List of possible date formats
    date_formats = ["%d-%b-%Y", "%d %b %Y", "%Y-%m-%d %H:%M:%S.%f%z"]

    for date_format in date_formats:
        try:
            dt = datetime.strptime(date_string, date_format)

            # If the parsed datetime doesn't have timezone info, assume it's UTC
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=pytz.UTC)
            else:
                # If it has timezone info, convert to UTC
                dt = dt.astimezone(pytz.UTC)

            if eod:
                # Set time to 3:30 PM India time for all formats when eod is True
                india_time = time(hour=15, minute=30)
                india_tz = pytz.timezone("Asia/Kolkata")
                dt = india_tz.localize(datetime.combine(dt.date(), india_time))
                dt = dt.astimezone(pytz.UTC)
            elif dt.time() == time(0, 0):  # If time is midnight (00:00:00)
                # Keep it as midnight UTC
                dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)

            return dt
        except ValueError:
            continue

    # If none of the formats work, raise an error
    raise ValueError(f"Unable to parse date string: {date_string}")


def convert_to_numeric(col: pd.Series):
    """convert to numeric if possible, only for object dtypes"""

    if col.dtype == "object":
        try:
            return pd.to_numeric(col)
        except ValueError:
            return col
    return col


def convert_daily_volatility_to_yearly(daily_volatility, days: float = 252):
    return daily_volatility * math.sqrt(days)


def equity_iv_df(quotes: dict) -> pd.DataFrame:
    """Build a core df with symbol, undPrice, expiry, strike, volatilities, lot and price."""

    flat_data = json_normalize(quotes, sep="-")

    # get lot from quote
    lot = (
        quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")
    )

    # build the df
    df = pd.DataFrame(flat_data)

    df = pd.DataFrame(
        [
            {
                "symbol": symbol,
                "instrument": quotes.get("stocks")[i]
                .get("metadata")
                .get("instrumentType"),
                "undPrice": stock_price,
                "expiry": quotes.get("stocks")[i].get("metadata").get("expiryDate"),
                "strike": quotes.get("stocks")[i].get("metadata").get("strikePrice"),
                "hv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("annualisedVolatility"),
                "iv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("impliedVolatility"),
                "lot": lot,
                "price": quotes.get("stocks")[i].get("metadata").get("lastPrice"),
            }
            for i in range(len(quotes))
        ]
    )

    # Convert expiry to UTC NSE eod
    df = df.assign(
        expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True))
    )

    # Convert the rest to numeric
    df = df.apply(convert_to_numeric)

    # Change instrument type
    instrument_dict = {
        "Stock": "STK",
        "Options": "OPT",
        "Currency": "FX",
        "Index": "IDX",
        "Futures": "FUT",
    }

    inst = df.instrument.str.split()

    s = inst.apply(lambda x: "".join(instrument_dict[item] for item in x))

    df = df.assign(instrument=s)

    return df


def find_closest_strike(df, above=False):
    """
    Finds the row with the strike closest to the undPrice.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    above (bool): If True, find the closest strike above undPrice. If False, find the closest strike below undPrice.

    Returns:
    pd.DataFrame: A DataFrame with the single row that has the closest strike.
    """
    undPrice = df["undPrice"].iloc[0]  # Get the undPrice from the first row

    if above:
        # Filter for strikes above undPrice
        mask = df["strike"] > undPrice
    else:
        # Filter for strikes below undPrice
        mask = df["strike"] < undPrice

    if not mask.any():
        return pd.DataFrame()  # Return an empty DataFrame if no rows match the criteria

    # Calculate the absolute difference between strike and undPrice
    diff = np.abs(df.loc[mask, "strike"] - undPrice)

    # Find the index of the row with the minimum difference
    closest_index = diff.idxmin()

    # Return the closest row as a DataFrame
    return df.loc[[closest_index]]


def get_dte(s: pd.Series) -> pd.Series:
    """Gets days to expiry. Expects series of UTC timestamps"""

    now_utc = datetime.now(pytz.UTC)
    return (s - now_utc).dt.total_seconds() / (24 * 60 * 60)


def fbfillnas(ser: pd.Series) -> pd.Series:
    """Fills nan in series forwards first and then backwards"""

    s = ser.copy()

    # Find the first non-NaN value
    first_non_nan = s.dropna().iloc[0]

    # Fill first NaN with the first non-NaN value
    s.iloc[0] = first_non_nan

    # Fill remaining NaN values with the next valid value
    s = s.fillna(s.bfill())

    # Fill remaining NaN values with the previous valid value
    s = s.fillna(s.ffill())

    return ser.fillna(s)


def get_a_stdev(iv: float, price: float, dte: float) -> float:
    """Gives 1 Standard Deviation value for annual iv"""
    
    return iv * price * math.sqrt(dte / 365)

# Classes

In [ ]:
# CLASSES
# *******


class Stocks:
    time_out = 5
    base_url = "https://www.nseindia.com/api"
    page_url = "https://www.nseindia.com/get-quotes/equity?symbol=LT"
    _routes = {
        "stock_meta": "/equity-meta-info",
        "stock_quote": "/quote-equity",
        "stock_derivative_quote": "/quote-derivative",
        "market_status": "/marketStatus",
        "chart_data": "/chart-databyindex",
        "market_turnover": "/market-turnover",
        "equity_derivative_turnover": "/equity-stock",
        "all_indices": "/allIndices",
        "live_index": "/equity-stockIndices",
        "index_option_chain": "/option-chain-indices",
        "equity_option_chain": "/option-chain-equities",
        "currency_option_chain": "/option-chain-currency",
        "pre_open_market": "/market-data-pre-open",
        "holiday_list": "/holiday-master?type=trading",
        "stock_history": "/historical/cm/equity",  # added by rkv
    }

    def __init__(self):
        self.s = requests.Session()
        h = {
            "Host": "www.nseindia.com",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
            "pragma": "no-cache",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
        self.s.headers.update(h)
        self.s.get(self.page_url)

    def get(self, route, payload={}):
        url = self.base_url + self._routes[route]
        r = self.s.get(url, params=payload)
        return r.json()

    @live_cache
    def stock_quote(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_quote", data)

    @live_cache
    def stock_quote_fno(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_derivative_quote", data)

    @live_cache
    def trade_info(self, symbol):
        data = {"symbol": symbol, "section": "trade_info"}
        return self.get("stock_quote", data)

    @live_cache
    def market_status(self):
        return self.get("market_status", {})

    @live_cache
    def chart_data(self, symbol, indices=False):
        data = {"index": symbol + "EQN"}
        if indices:
            data["index"] = symbol
            data["indices"] = "true"
        return self.get("chart_data", data)

    @live_cache
    def tick_data(self, symbol, indices=False):
        return self.chart_data(symbol, indices)

    @live_cache
    def market_turnover(self):
        return self.get("market_turnover")

    @live_cache
    def eq_derivative_turnover(self, type="allcontracts"):
        data = {"index": type}
        return self.get("equity_derivative_turnover", data)

    @live_cache
    def all_indices(self):
        return self.get("all_indices")

    def live_index(self, symbol="NIFTY 50"):
        data = {"index": symbol}
        return self.get("live_index", data)

    @live_cache
    def index_option_chain(self, symbol="NIFTY"):
        data = {"symbol": symbol}
        return self.get("index_option_chain", data)

    @live_cache
    def equities_option_chain(self, symbol):
        data = {"symbol": symbol}
        return self.get("equity_option_chain", data)

    @live_cache
    def currency_option_chain(self, symbol="USDINR"):
        data = {"symbol": symbol}
        return self.get("currency_option_chain", data)

    @live_cache
    def live_fno(self):
        return self.live_index("SECURITIES IN F&O")

    @live_cache
    def pre_open_market(self, key="NIFTY"):
        data = {"key": key}
        return self.get("pre_open_market", data)

    @live_cache
    def holiday_list(self):
        return self.get("holiday_list", {})

    @live_cache
    def stock_history(self, symbol, days: int = 365, chunks: int = 50):

        date_ranges = make_date_range_for_stock_history(symbol, days, chunks)

        result = []
        for dr in date_ranges:
            result.append(self.get("stock_history", dr))

        df = clean_stock_history(result)

        return df


class IDXHistories:

    time_out = 5
    base_url = "https://niftyindices.com"
    url = "https://niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString"

    # prepare `post` header
    post_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.77 Safari/537.36",
        "Connection": "keep-alive",
        "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "DNT": "1",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua-mobile": "?0",
        "Content-Type": "application/json; charset=UTF-8",
        "Origin": "https://niftyindices.com",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Dest": "empty",
        "Referer": "https://niftyindices.com/reports/historical-data",
        "Accept-Language": "en-US,en;q=0.9,hi;q=0.8",
    }

    def __init__(self, days: int = 365) -> None:
        self.s = requests.Session()

        # update session with default headers and get the cookies
        init_header = requests.utils.default_headers()
        init_header.update(
            {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.77 Safari/537.36",
            }
        )
        self.s.headers.update(init_header)
        c = self.s.get(url=self.url)
        self.cookies = c.cookies

    def get(self, payload={}):

        r = self.s.post(
            url=self.url,
            headers=self.post_header,
            cookies=self.cookies,
            data=payload,
            timeout=self.time_out,
        )

        return r.json()

    def make_histories(self, days: int = 365, chunks: int = 50):
        """Makes histories for NIFTY50 and BANKNIFTY, based on number of days provided"""

        date_ranges = split_dates(days=days, chunks=chunks)

        idx_symbols = ["Nifty Bank", "Nifty 50"]

        # organize the payloads
        payloads = [
            {
                "cinfo": str(
                    {
                        "name": idx_symbol,
                        "startDate": s.strftime("%d-%b-%Y"),
                        "endDate": e.strftime("%d-%b-%Y"),
                        "indexName": idx_symbol,
                    }
                )
            }
            for s, e in date_ranges
            for idx_symbol in idx_symbols
        ]
        # get the raw jsons
        results = []

        for payload in tqdm(payloads):
            r = self.get(payload=json.dumps(payload))
            results.append(r)

        df = clean_index_history(results)

        return df

# Stock FnOs

## Initialize `Stocks` class

In [ ]:
nse = Stocks()

## Get underlying price

In [ ]:
symbol = "SBIN"

q = nse.stock_quote(symbol)
stock_price = q.get("priceInfo").get("lastPrice")

## Get equity fno list

In [ ]:
equities = nse.live_fno()

# Equities set
fno_equities = {kv.get("symbol") for kv in equities.get("data")}

## Get a stock quote

In [ ]:
quotes = nse.stock_quote_fno(symbol)

### Get lot from a stock quote

In [ ]:
lot = quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")

### Get hv and iv with lot from fno quotes

In [ ]:
df_vola = equity_iv_df(quotes)

In [ ]:
df_vola.head()

### Find strike closest to underlying

In [ ]:
closest_to_und = find_closest_strike(df_vola, above=False)
closest_to_und

### Get underlying volatlities

In [ ]:
volatilities = next(iter(quotes.get("underlyingInfo").get("volatility")))

und_vols = {
    k: convert_daily_volatility_to_yearly(float(v)) for k, v in volatilities.items()
}

und_vols

# Stock History

## Get Stock History

In [ ]:
df = nse.stock_history("ongc", days=365, chunks=50)

df.head()

# Index History
 - Index histories are treated in a separate class as:
     - its URL (https://niftyindices.com) is different than equity
     - which needs a POST after of a GET request
     - and also has its date string with a dash (20-Jul-2024)

## Initialize Index Histories

In [ ]:
idx = IDXHistories()

df_idx_hist = idx.make_histories(20)  # give number of days needed

df_idx_hist.groupby("symbol").head(3)

# Get All chains

In [ ]:
stock_chain = nse.equities_option_chain(symbol)  # Equity option chains
data = stock_chain.get("records").get("data")

# idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
# data = idx_chain.get('records').get('data')

pe = [data[i].get("PE") for i in range(len(data))]
df_pe = json_normalize(pe, sep="-").dropna(subset=["identifier"])
# df_pe = df_pe.sort_values('strikePrice', ascending=False).reset_index(drop=True)
df_pe["right"] = "P"

ce = [data[i].get("CE") for i in range(len(data))]
df_ce = json_normalize(ce, sep="-").dropna(subset=["identifier"])
df_ce["right"] = "C"
# df_ce = df_ce.sort_values('strikePrice', ascending=False).reset_index(drop=True)

df = pd.concat([df_ce, df_pe], ignore_index=True)

chain_col_map = {
    "underlying": "symbol",
    "underlyingValue": "undPrice",
    "expiryDate": "expiry",
    "strikePrice": "strike",
    "right": "right",
    "openInterest": "oi",
    "changeinOpenInterest": "oi_chg",
    "pchangeinOpenInterest": "oi_pchg",
    "totalTradedVolume": "volume",
    "change": "change",
    "pChange": "ltp_pct_chg",
    "totalBuyQuantity": "buy_qty",
    "totalSellQuantity": "sell_qty",
    "bidQty": "bid_qty",
    "bidprice": "bid",
    "askQty": "ask_qty",
    "askPrice": "ask",
    "impliedVolatility": "iv",
    "dte": "dte",
    "stdev": "stdev",
    "lastPrice": "ltp",
}

df.rename(columns=chain_col_map, inplace=True, errors="ignore")

# Convert expiry to UTC NSE eod
df = df.assign(expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True)))

# Convert the rest to numeric
df = df.apply(convert_to_numeric)

# Replace zero values of iv and ltp to np.nan
df["iv"] = df["iv"].replace(0, np.nan)
df["ltp"] = df["ltp"].replace(0, np.nan)

# Gets dte
df["dte"] = get_dte(df.expiry)

# Sort the columns
df = df.sort_values(
    ["dte", "right", "strike"], ascending=[True, True, False]
).reset_index(drop=True)

# Fill missing ivs
df.iv = fbfillnas(df.iv)

# Get standard deviations of strike from underlying.
## ... NOTE: iv is divided by 100 to make it into %ge

stdev = df.apply(
    lambda row: get_a_stdev(iv=row["iv"]/100, price=row["undPrice"], dte=row["dte"]), axis=1
)

df = df.assign(stdev=stdev)

In [ ]:
def get_closest_values(myArr: list, 
                       myNumber: float, 
                       how_many: int=0):
    
    """Get closest values in a list
    
    how_many: 0 gives the closest value\n
              1 | 2 | ... use for CALL fences\n
             -1 | -2 | ... use for PUT fences"""

    i = 0

    result = []

    while i <= abs(how_many):

        if how_many > 0:
            # going right
            val = myArr[myArr > myNumber].min()
            
        elif how_many < 0:
            # going left
            val = myArr[myArr < myNumber].max()
            
        else:
            val = min(myArr.tolist(), key=lambda x: abs(x-myNumber))

        result.append(val)
        myNumber = val
        i +=1

    output = result[0:abs(1 if how_many == 0 else abs(how_many))]

    return output

In [ ]:
dfc = df[df.right == 'C']
closest_devs = dfc.groupby('dte').stdev.apply(lambda x: get_closest_values(x, CALLSTDMULT, 1)[0])
closest_devs

In [ ]:
dfcs = dfc[dfc.stdev.isin(closest_devs.to_list())]

In [ ]:
# Choose columns to show
chain_cols = [v for _, v in chain_col_map.items() if v in df.columns]
dfcs[chain_cols]

In [ ]:
strike = list(range(100, 150, 5))[1:]*2

In [ ]:
dte = [5, 10]*int(len(strike)/2)

In [ ]:
size = len(strike)
random_numbers = np.random.rand(size)

nan_probability = 0.3
nan_mask = np.random.random(size) < nan_probability
iv = np.where(nan_mask, np.nan, random_numbers)

p_choice = 0.5
c_choice = 1 - p_choice

right = np.random.choice(['P', 'C'], size, p=[p_choice, c_choice])

marker = 0.3


In [ ]:
df = pd.DataFrame({'strike': strike, 'dte': dte, 'iv': iv, 'right': right})

In [ ]:
def choose_closest_iv(df, marker):
  """Chooses the closest iv value to marker for each group in df.

  Args:
      df (pd.DataFrame): Dataframe containing columns 'strike', 'dte', 'iv', and 'right'.
      marker (float): The value to which the closest iv should be chosen.

  Returns:
      pd.DataFrame: A new DataFrame with the same columns as the input df,
                    but with the 'iv' column replaced by the closest values.
  """

  def g(group):
    """Function applied to each group in df.

    Args:
        group (pd.DataFrame): A subgroup of df based on 'dte' and 'right'.

    Returns:
        pd.DataFrame: The group with the 'iv' column replaced by the closest values.
    """
    if group['right'].iloc[0] == 'P':
      # For Put options, choose the closest value below marker
      return group[group['iv'] <= marker].sort_values(by='iv', ascending=False).head(1)
    else:
      # For Call options, choose the closest value above marker
      return group[group['iv'] >= marker].sort_values(by='iv', ascending=True).head(1)

  return df.groupby(['dte', 'right']).apply(g)


In [ ]:
choose_closest_iv(df.copy(), marker)

# .... STOPPED HERE for `RIM` options

In [ ]:
idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
curr_option_chain = nse.currency_option_chain("USDINR")  # Currency option chains

# Option Chains

# Miscellaneous

## Convert nse to ib friendly symbols

In [ ]:
raw_fnos = nse.live_fno()
fno_list = {data.get("symbol") for data in raw_fnos.get("data")}